# Using pretrained models (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatibl

In [2]:
from datasets import load_dataset

address_ner_dataset = load_dataset("TrevorJS/synth-addresses-ner-mk3")

#from transformers import AutoTokenizer, AutoModelForTokenClassification
#from transformers import pipeline

#tokenizer = AutoTokenizer.from_pretrained("dslim/distilbert-NER")

#def tokenize_function(examples):
#    return tokenizer(examples["ocr_text"], padding="max_length", truncation=True)

#tokenized_datasets = address_ner_dataset.map(tokenize_function, batched=True)

#small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
#small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/151M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/151M [00:00<?, ?B/s]

eval-00000-of-00001.parquet:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/112M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1350000 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500000 [00:00<?, ? examples/s]

In [7]:
print(address_ner_dataset['train'].features)

{'name': Value(dtype='string', id=None), 'state_name': Value(dtype='string', id=None), 'state_abbreviation': Value(dtype='string', id=None), 'postal_code': Value(dtype='string', id=None), 'city': Value(dtype='string', id=None), 'street_number': Value(dtype='string', id=None), 'street_name': Value(dtype='string', id=None), 'unit_number': Value(dtype='string', id=None), 'unit_designator': Value(dtype='string', id=None), 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ocr_text': Value(dtype='string', id=None), 'type': Value(dtype='string', id=None), 'tracking_number': Value(dtype='string', id=None), 'plus_4': Value(dtype='string', id=None)}


In [3]:
# Accessing the label names from the 'ner_tags' feature.
label_names = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'] #address_ner_dataset['train'][0]['ner_tags'] #tokenized_datasets['train'].features['ner_tags']
label_names

['B-NAME',
 'I-NAME',
 'B-STREET_NUMBER',
 'B-STREET_NAME',
 'I-STREET_NAME',
 'O',
 'O',
 'O',
 'B-CITY',
 'O',
 'B-STATE_ABBREVIATION',
 'B-POSTAL_CODE',
 'O',
 'B-PLUS_4']

In [4]:
address_ner_dataset['train'][0]

{'name': 'Gemmi Batteen',
 'state_name': 'Nebraska',
 'state_abbreviation': 'NE',
 'postal_code': '69351',
 'city': 'Lakeside',
 'street_number': '39',
 'street_name': 'Tara Court',
 'unit_number': None,
 'unit_designator': None,
 'tokens': ['Gemmi',
  'Batteen',
  '39',
  'Tara',
  'Court',
  'Adult',
  'Signature',
  'Required',
  'Lakeside',
  ',',
  'NE',
  '69351',
  '-',
  '1146'],
 'ner_tags': ['B-NAME',
  'I-NAME',
  'B-STREET_NUMBER',
  'B-STREET_NAME',
  'I-STREET_NAME',
  'O',
  'O',
  'O',
  'B-CITY',
  'O',
  'B-STATE_ABBREVIATION',
  'B-POSTAL_CODE',
  'O',
  'B-PLUS_4'],
 'ocr_text': 'Gemmi Batteen 39 Tara Court Adult Signature Required Lakeside , NE 69351-1146',
 'type': 'individual',
 'tracking_number': None,
 'plus_4': '1146'}

In [5]:
from transformers import AutoTokenizer

# Define the checkpoint you want to use for the tokenizer.
checkpoint = 'dslim/distilbert-NER'

# Create a tokenizer instance by loading the pre-trained checkpoint.
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [6]:
# Tokenize the first training example from the dataset
token = tokenizer(address_ner_dataset['train'][0]['tokens'], is_split_into_words = True)

# Print the tokenizer object, the tokenized tokens, and the word IDs
print(token, '\n--------------------------------------------------------------------------------------\n',
      token.tokens(),'\n--------------------------------------------------------------------------------------\n',
      token.word_ids())

{'input_ids': [101, 144, 5521, 3080, 21928, 10381, 3614, 10235, 2031, 11635, 20979, 17337, 11336, 15174, 1181, 10180, 3269, 117, 26546, 5691, 19297, 1475, 118, 12620, 1545, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]} 
--------------------------------------------------------------------------------------
 ['[CLS]', 'G', '##em', '##mi', 'Bat', '##teen', '39', 'Tara', 'Court', 'Adult', 'Sign', '##ature', 'Re', '##quire', '##d', 'Lakes', '##ide', ',', 'NE', '69', '##35', '##1', '-', '114', '##6', '[SEP]'] 
--------------------------------------------------------------------------------------
 [None, 0, 0, 0, 1, 1, 2, 3, 4, 5, 6, 6, 7, 7, 7, 8, 8, 9, 10, 11, 11, 11, 12, 13, 13, None]


In [8]:
def align_target(labels, word_ids):
    # Define a mapping from beginning (B-) labels to inside (I-) labels
    begin2inside = {
        "B-NAME": "I-NAME",  # B-LOC -> I-LOC
        "B-STREET_NUMBER": "I-STREET_NUMBER",  # B-MISC -> I-MISC
        "B-STREET_NAME": "I-STREET_NAME",  # B-ORG -> I-ORG
        "B-UNIT_NUMBER": "I-UNIT_NUMBER",    # B-PER -> I-PER
        "B-UNIT_DESIGNATOR" : "I-UNIT_DESIGNATOR",
        "B-CITY" : " I-CITY",
        "B-STATE_ABBREVIATION" : "I-STATE_ABBREVIATION",
        "B-PLUS_4" : "I-PLUS_4",
        "B-STATE_NAME" : "I-STATE_NAME",
        "B-POSTAL_CODE" : "I-POSTAL_CODE",
    }

    # Initialize an empty list to store aligned labels and a variable to track the last word
    align_labels = []
    last_word = None

    # Iterate through the word_ids
    for word in word_ids:
        if word is None:
            label = -100  # Set label to -100 for None word_ids
        elif word != last_word:
            label = labels[word]  # Use the label corresponding to the current word_id
        else:
            label = labels[word]
            # Change B- to I- if the previous word is the same
            if label in begin2inside:
                label = begin2inside[label]  # Map B- to I-

        # Append the label to the align_labels list and update last_word
        align_labels.append(label)
        last_word = word

    return align_labels

In [9]:
# Extract labels and word_ids
labels = address_ner_dataset['train'][0]['ner_tags']
word_ids = token.word_ids()

# Use the align_target function to align labels
aligned_target = align_target(labels, word_ids)

# Print tokenized tokens, original labels, and aligned labels
print(token.tokens(), '\n--------------------------------------------------------------------------------------\n',
      labels, '\n--------------------------------------------------------------------------------------\n',
      aligned_target)

['[CLS]', 'G', '##em', '##mi', 'Bat', '##teen', '39', 'Tara', 'Court', 'Adult', 'Sign', '##ature', 'Re', '##quire', '##d', 'Lakes', '##ide', ',', 'NE', '69', '##35', '##1', '-', '114', '##6', '[SEP]'] 
--------------------------------------------------------------------------------------
 ['B-NAME', 'I-NAME', 'B-STREET_NUMBER', 'B-STREET_NAME', 'I-STREET_NAME', 'O', 'O', 'O', 'B-CITY', 'O', 'B-STATE_ABBREVIATION', 'B-POSTAL_CODE', 'O', 'B-PLUS_4'] 
--------------------------------------------------------------------------------------
 [-100, 'B-NAME', 'I-NAME', 'I-NAME', 'I-NAME', 'I-NAME', 'B-STREET_NUMBER', 'B-STREET_NAME', 'I-STREET_NAME', 'O', 'O', 'O', 'O', 'O', 'O', 'B-CITY', ' I-CITY', 'O', 'B-STATE_ABBREVIATION', 'B-POSTAL_CODE', 'I-POSTAL_CODE', 'I-POSTAL_CODE', 'O', 'B-PLUS_4', 'I-PLUS_4', -100]


In [10]:
# Create a list of aligned labels using label names
aligned_labels = aligned_target

# Loop through tokens and aligned labels and print them
for x, y in zip(token.tokens(), aligned_labels):
    print(f"{x}\t{y}")

[CLS]	-100
G	B-NAME
##em	I-NAME
##mi	I-NAME
Bat	I-NAME
##teen	I-NAME
39	B-STREET_NUMBER
Tara	B-STREET_NAME
Court	I-STREET_NAME
Adult	O
Sign	O
##ature	O
Re	O
##quire	O
##d	O
Lakes	B-CITY
##ide	 I-CITY
,	O
NE	B-STATE_ABBREVIATION
69	B-POSTAL_CODE
##35	I-POSTAL_CODE
##1	I-POSTAL_CODE
-	O
114	B-PLUS_4
##6	I-PLUS_4
[SEP]	-100


In [33]:
def tokenize_fn(batch):
    # Tokenize the input batch
    tokenized_inputs = tokenizer(batch['tokens'], truncation=True, is_split_into_words=True, padding="max_length")

    # Extract the labels batch from the input batch
    labels_batch = batch['ner_tags']

    """   # Initialize a list to store aligned targets for each example in the batch
    aligned_targets_batch = []

    # Iterate through each example and align the labels
    for i, labels in enumerate(labels_batch):
        # Extract the word_ids for the current example
        word_ids = tokenized_inputs.word_ids(i)

        # Use the align_target function to align the labels
        aligned_targets_batch.append(align_target(labels, word_ids)) """

    # Add the aligned labels to the tokenized inputs under the key "labels"
    tokenized_inputs["labels"] = labels_batch

    # Return the tokenized inputs, including aligned labels
    return tokenized_inputs

In [34]:
tokenized_dataset = address_ner_dataset.map(tokenize_fn, batched=True, remove_columns=address_ner_dataset['train'].column_names)

Map:   0%|          | 0/1350000 [00:00<?, ? examples/s]

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500000 [00:00<?, ? examples/s]

In [35]:
from transformers import DataCollatorForTokenClassification

# Create a DataCollatorForTokenClassification object
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
#print(tokenized_dataset['train'])
# Testing data using the data collator
#batch = data_collator([tokenized_dataset['train'][i] for i in range(1)])

# Display the resulting batch
#batch

In [13]:
# Install the seqeval library for evaluating sequence tasks
!pip install seqeval ;
!pip install evaluate ;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=366eb86321d5681165e41715a91dcb2f1c169ac85e6717d2d7119a26e39da099
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [36]:
# Import the seqeval metric from Hugging Face's datasets library
import evaluate

# Load the seqeval metric which can evaluate NER and other sequence tasks
metric = evaluate.load("seqeval")

# Example usage: compute metric on a sample predictions and reference list
# predictions and references should be a list of lists containing predicted and true token labels

# List of List Input
metric.compute(predictions = [['O' , 'B-ORG' , 'I-ORG']],
               references = [['O' , 'B-MISC' , 'I-ORG']])

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'MISC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'ORG': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.6666666666666666}

In [37]:
import numpy as np
# Function to compute evaluation metrics from model logits and true labels
def compute_metrics(logits_and_labels):

  # Unpack the logits and labels
  logits, labels = logits_and_labels

  # Get predictions from the logits
  predictions = np.argmax(logits, axis=-1)

  # Remove ignored index (special tokens)
  str_labels = [
    [label_names[t] for t in label if t!=-100] for label in labels
  ]

  str_preds = [
    [label_names[p] for (p, t) in zip(prediction, label) if t != -100]
    for prediction, label in zip(predictions, labels)
  ]

  # Compute metrics
  results = metric.compute(predictions=str_preds, references=str_labels)

  # Extract key metrics
  return {
    "precision": results["overall_precision"],
    "recall": results["overall_recall"],
    "f1": results["overall_f1"],
    "accuracy": results["overall_accuracy"]
  }

In [38]:
# Create mapping from label ID to label string name
id2label = {k: v for k, v in enumerate(label_names)}

# Create reverse mapping from label name to label ID
label2id = {v: k for k, v in enumerate(label_names)}

print(id2label , '\n--------------------\n' , label2id)
print(label_names)

{0: 'B-NAME', 1: 'I-NAME', 2: 'B-STREET_NUMBER', 3: 'B-STREET_NAME', 4: 'I-STREET_NAME', 5: 'O', 6: 'O', 7: 'O', 8: 'B-CITY', 9: 'O', 10: 'B-STATE_ABBREVIATION', 11: 'B-POSTAL_CODE', 12: 'O', 13: 'B-PLUS_4'} 
--------------------
 {'B-NAME': 0, 'I-NAME': 1, 'B-STREET_NUMBER': 2, 'B-STREET_NAME': 3, 'I-STREET_NAME': 4, 'O': 12, 'B-CITY': 8, 'B-STATE_ABBREVIATION': 10, 'B-POSTAL_CODE': 11, 'B-PLUS_4': 13}
['B-NAME', 'I-NAME', 'B-STREET_NUMBER', 'B-STREET_NAME', 'I-STREET_NAME', 'O', 'O', 'O', 'B-CITY', 'O', 'B-STATE_ABBREVIATION', 'B-POSTAL_CODE', 'O', 'B-PLUS_4']


In [39]:
# Load pretrained token classification model from Transformers
from transformers import AutoModelForTokenClassification

# Initialize model object with pretrained weights
model = AutoModelForTokenClassification.from_pretrained(checkpoint)

In [40]:
# Configure training arguments using TrainigArguments class
from transformers import TrainingArguments

training_args = TrainingArguments(
  # Location to save fine-tuned model
  output_dir = "address_parser_fine_tuned_model",

  # Evaluate each epoch
  evaluation_strategy = "epoch",

  # Learning rate for Adam optimizer
  learning_rate = 2e-5,

  # Batch sizes for training and evaluation
  per_device_train_batch_size = 16,
  per_device_eval_batch_size = 16,

  # Number of training epochs
  num_train_epochs = 3,

  # L2 weight decay regularization
  weight_decay = 0.01
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [45]:
# Initialize Trainer object for model training
from transformers import Trainer

trainer = Trainer(
  # Model to train
  model=model,

  # Training arguments
  args=training_args,

  # Training and validation datasets
  train_dataset=tokenized_dataset["train"].shuffle(seed=42).select(range(100)),
  eval_dataset=tokenized_dataset["test"].shuffle(seed=42).select(range(100)),

  # Tokenizer
  tokenizer=tokenizer,

  # Custom metric function
  compute_metrics=compute_metrics,

  # Data collator
  #data_collator=data_collator
)

<ipython-input-45-a22d0658e8a0>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [46]:
trainer.train()

ValueError: too many dimensions 'str'